# Standardized Approach - Counterparty Credit Risk



In [1]:
# Librerias necesarias
import math
from math import exp
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import norm

In [2]:
# Formato CSV esperado
# ------------------------------------------------------------
# Columnas recomendadas (una fila por transacción):
# - netting_set_id : identificador del netting set (string)
# - trade_id       : identificador de la transacción (string)
# - asset_class    : 'IR','FX','Credit','Equity','Commodity' (string)
# - notional       : cantidad de notional (en unidades de la columna currency_dom)
# - currency       : moneda del notional (p. ej. 'USD')
# - foreign_leg_notional : (solo FX) notional de la "otra" pierna en su moneda original
# - price          : (Equity/Commodity) precio por unidad (o NaN)
# - quantity       : (Equity/Commodity) número de unidades (o NaN)
# - start_date     : (IR/Credit) fecha de inicio del periodo referenciado, formato ISO 'YYYY-MM-DD' o NaN
# - end_date       : (IR/Credit) fecha final del periodo referenciado, formato ISO 'YYYY-MM-DD' o NaN
# - maturity_date  : fecha de madurez del contrato (para calcular M_i) formato ISO 'YYYY-MM-DD'
# - is_option      : True/False
# - option_type    : 'call' o 'put' (solo si is_option)
# - option_strike  : strike (solo si is_option)
# - option_forward_price : forward/spot price used as underlying reference (P or F) (solo si is_option)
# - position       : 'long' o 'short' (respecto al instrumento)
# - credit_rating  : rating de la referencia (solo Credit: 'AAA','AA','A','BBB','BB','B','CCC' o 'IG'/'SG' para índices)
# - margin_flag    : True si el netting set está bajo acuerdo de margen (esto se puede proporcionar a nivel netting_set)
#
# Además se espera un segundo CSV o un diccionario (por netting set) con parámetros de margen/colateral:
# - netting_set_id
# - collateral_value (C) : valor neto del colateral actualmente en poder del banco (incluye VM + IM cuando corresponda)\# - threshold (TH) : threshold de la contraparte (en la misma moneda)
# - mta : minimum transfer amount (MTA)
# - nica: net independent collateral amount (NICA)
# - collateral_currency : moneda de collateral (para convertir si es necesario)
# - alpha : factor (por defecto 1.4)
# - domestic_currency : moneda de reporte (p. ej. 'USD')

In [3]:
# PARTE 1: Parámetros supervisorios (Tabla 2 del documento)

SUPERVISORY_FACTORS = {
    'IR': 0.005,        # 0.50%
    'FX': 0.04,         # 4.0%
    # Credit single-name por rating (en decimal): AAA, AA -> 0.0038 etc.
    'Credit_single': {
        'AAA': 0.0038, 'AA': 0.0038, 'A': 0.0042, 'BBB': 0.0054,
        'BB': 0.0106, 'B': 0.016, 'CCC': 0.06
    },
    'Credit_index': {'IG':0.0038, 'SG':0.0106},
    'Equity_single': 0.32,
    'Equity_index': 0.20,
    'Commodity': {
        'Electricity': 0.40,
        'Oil/Gas': 0.18,
        'Metals': 0.18,
        'Agricultural': 0.18,
        'Other': 0.18
    }
}

# Supervisory option volatilities (Table 2)
SUPERVISORY_OPTION_VOL = {
    'IR': 0.50,
    'FX': 0.15,
    'Credit_single': 1.00,
    'Credit_index': 0.80,
    'Equity_single': 1.20,
    'Equity_index': 0.75,
    'Commodity': {
        'Electricity': 1.50,
        'Oil/Gas': 0.70,
        'Metals': 0.70,
        'Agricultural': 0.70,
        'Other': 0.70
    }
}

# Correlation parameters for single-name vs index (Table 2)
CORRELATION_PARAMS = {
    'Credit_single': 0.5,
    'Credit_index': 0.8,
    'Equity_single': 0.5,
    'Equity_index': 0.8,
    'Commodity_hedging_set': 0.4  # dentro de un hedging set (energy/metals/..)
}

# Otros parámetros
ALPHA = 1.4  # factor alfa
FLOOR = 0.05 # floor para multiplier
BUSINESS_DAYS_PER_YEAR = 250.0
MIN_MATURITY_DAYS_FLOOR = 10.0  # El documento define piso de 10 días hábiles para iS floored


In [5]:
# %%
# Funciones auxiliares: fechas -> años, duración supervisoria, maturity factors, MPOR

def year_fraction(date_from, date_to):
    """Devuelve la fracción de años entre dos fechas, usando 250 días hábiles por año.
    Acepta strings ISO o datetime. Si alguno es NaN/None devuelve None.
    """
    if pd.isna(date_from) or pd.isna(date_to):
        return None
    if isinstance(date_from, str):
        date_from = datetime.fromisoformat(date_from)
    if isinstance(date_to, str):
        date_to = datetime.fromisoformat(date_to)
    delta_days = (date_to - date_from).days
    return max(delta_days / BUSINESS_DAYS_PER_YEAR, 0.0)


def supervisory_duration(S_years, E_years):
    """Cálculo de la duración supervisoria:
    SD_i = (exp(-0.05 * S_i) - exp(-0.05 * E_i)) / 0.05
    S_years: start in years (floored por 10 días hábiles si corresponde)
    E_years: end in years
    """
    # Aplicar piso S >= 10 business days
    floor_years = MIN_MATURITY_DAYS_FLOOR / BUSINESS_DAYS_PER_YEAR
    if S_years is None:
        S_years = 0.0
    S = max(S_years, floor_years)
    E = max(E_years if E_years is not None else S, S)  # garantizar E >= S
    sd = (math.exp(-0.05 * S) - math.exp(-0.05 * E)) / 0.05
    return sd


def maturity_factor_unmargined(M_years):
    """
    Maturity factor para transacciones unmargined:
    MF_unmargined = min(1, sqrt( M_i (años) / 1 year ))
    donde M_i está floored por 10 business days (ver párrafos del documento).
    """
    floor_years = MIN_MATURITY_DAYS_FLOOR / BUSINESS_DAYS_PER_YEAR
    M = max(M_years if M_years is not None else 0.0, floor_years)
    mf = min(1.0, math.sqrt(M / 1.0))
    return mf


def maturity_factor_margined(MPOR_days):
    """
    Maturity factor para transacciones margined (re-escalado por MPOR):
    MF_margined = sqrt( (3/2) * MPOR / 250 )
    (parrafo 164)
    """
    if MPOR_days is None or MPOR_days <= 0:
        return 1.0
    mf = math.sqrt((1.5 * MPOR_days) / BUSINESS_DAYS_PER_YEAR)
    return mf


def compute_mpor_from_margin_frequency(frequency_days):
    """
    Si hay remarginación periódica cada N días, MPOR = 10 + N - 1 (párrafo del anexo).
    Si daily remargining -> MPOR = 10 (por convención en el documento).
    """
    if frequency_days is None:
        return 10.0
    return 10.0 + frequency_days - 1.0

In [6]:
# Función para asignar buckets de madurez (interest rate / credit):
# buckets: 1 = <1 año, 2 = [1,5] años, 3 = >5 años

def assign_ir_bucket(E_years):
    if E_years is None:
        return 3
    if E_years < 1.0:
        return 1
    elif E_years <= 5.0:
        return 2
    else:
        return 3

In [7]:
# Supervisory delta (parrafo 159):
# - Si no es opción: delta = +1 (long) o -1 (short)
# - Para opciones se usa la fórmula con la distribucion normal (ver doc). Para calls:
#    delta_call = Phi( ln(F/K) / (sigma * sqrt(T)) + 0.5 * sigma * sqrt(T) )
#   para puts: delta_put = delta_call - 1
#   aplicar signo según la posición (long/short)

def supervisory_delta(row):
    """Calcula el delta supervisorio para una fila/trade.
    row debe contener: is_option(bool), option_type('call'/'put'), option_forward_price,
    option_strike, option_T (años hasta exercise), position ('long'/'short'), asset_class
    """
    pos = row.get('position', 'long')
    sign_pos = 1.0 if pos.lower() == 'long' else -1.0

    if not row.get('is_option', False):
        return sign_pos * 1.0

    # si es opción:
    asset = row.get('asset_class')
    sigma = None
    if asset == 'IR':
        sigma = SUPERVISORY_OPTION_VOL['IR']
    elif asset == 'FX':
        sigma = SUPERVISORY_OPTION_VOL['FX']
    elif asset == 'Credit':
        # en credit la volatilidad depende de single vs index; si rating contiene 'IG' usamos index
        rating = row.get('credit_rating', '')
        if rating in ('IG', 'SG'):
            sigma = SUPERVISORY_OPTION_VOL['Credit_index']
        else:
            sigma = SUPERVISORY_OPTION_VOL['Credit_single']
    elif asset == 'Equity':
        # si referencia a index o single se debe indicar. Supongamos que hay columna 'is_index'
        if row.get('is_index', False):
            sigma = SUPERVISORY_OPTION_VOL['Equity_index']
        else:
            sigma = SUPERVISORY_OPTION_VOL['Equity_single']
    elif asset == 'Commodity':
        # si hay tipo de commodity
        typ = row.get('commodity_type', 'Other')
        sigma = SUPERVISORY_OPTION_VOL['Commodity'].get(typ, 0.70)
    else:
        sigma = 1.0

    # parámetros de la opción:
    F = float(row.get('option_forward_price', row.get('underlying_forward', 0.0)))
    K = float(row.get('option_strike', 0.0))
    T = float(row.get('option_T', 0.0))
    if T <= 0 or sigma <= 0 or F <= 0 or K <= 0:
        # fallback prudente
        # si no hay datos suficientes, usar delta absoluto = 1 para lineales o 0 para opciones no definidas
        return sign_pos * 1.0

    sqrtT = math.sqrt(T)
    d_arg = math.log(F / K) / (sigma * sqrtT) + 0.5 * sigma * sqrtT
    call_delta = norm.cdf(d_arg)
    if row.get('option_type', 'call').lower() == 'call':
        delta = call_delta
    else:
        # put parity: delta_put = call_delta - 1
        delta = call_delta - 1.0

    return sign_pos * delta

In [8]:
# Cálculo del adjusted notional por trade (parrafo 157-158).

def trade_adjusted_notional(row, domestic_currency='USD'):
    """Devuelve el adjusted notional (antes de aplicar delta y MF) para un trade.
    Para IR y Credit: notional convertido * supervisory_duration
    Para FX: notional de la foreign leg convertido
    Para Equity y Commodity: price * quantity
    """
    asset = row.get('asset_class')
    notional = float(row.get('notional', 0.0))

    # M_i: maturity in years = (maturity_date - today) / 250
    # En la práctica, usamos la fecha de madurez relativa a la fecha de cálculo;
    # aquí asumimos que la DataFrame nos da directamente 'maturity_years' para simplicidad.
    # Si solo se proporcionan fechas, el usuario puede calcular y añadir 'maturity_years' al CSV.
    M_years = row.get('maturity_years')

    if asset == 'IR' or asset == 'Credit':
        # calcular SD a partir de start/end (en años). Si start date ya pasó, S=0.
        S = row.get('start_years', 0.0)
        E = row.get('end_years', None)
        sd = supervisory_duration(S, E)
        adjusted = notional * sd
        return adjusted

    elif asset == 'FX':
        # adjusted notional = notional of foreign currency leg converted to domestic
        # se asume que 'foreign_leg_notional_dom' contiene el notional convertido a moneda doméstica
        foreign_dom = row.get('foreign_leg_notional_dom')
        if foreign_dom is not None:
            return float(foreign_dom)
        # fallback: usar provided notional
        return notional

    elif asset == 'Equity' or asset == 'Commodity':
        price = float(row.get('price', 0.0))
        qty = float(row.get('quantity', 0.0))
        return price * qty

    else:
        # si asset unknown, devolvemos notional
        return notional

In [9]:
# Cálculo del RC (Replacement Cost) por netting set (parrafo 136 y 144)
# RC for unmargined: RC = max( V - C, 0 ) where V = sum of market values
# RC for margined: RC = max( V - C - (TH + MTA - NICA), 0 )  -> equivalente a RC = max(V - C - (TH + MTA - NICA), 0)

def replacement_cost(netting_set_df, collateral_params):
    """Calcula RC para un netting set.
    netting_set_df: DataFrame solo para el netting set
    collateral_params: dict con keys: 'collateral_value', 'threshold', 'mta', 'nica'
    """
    V = netting_set_df['market_value'].sum()  # sum algebraic
    C = float(collateral_params.get('collateral_value', 0.0))
    TH = float(collateral_params.get('threshold', 0.0))
    MTA = float(collateral_params.get('mta', 0.0))
    NICA = float(collateral_params.get('nica', 0.0))
    margined = collateral_params.get('margined', False)

    if not margined:
        RC = max(V - C, 0.0)
    else:
        # RC = max(V - C - (TH + MTA - NICA), 0)
        RC = max(V - C - (TH + MTA - NICA), 0.0)
    return RC

In [10]:
# Funciones para calcular AddOn por asset class

# ADD-ON INTEREST RATE

def interest_rate_addon(netting_set_df, params_per_netting_set):
    """Calcula el add-on de interest rate para un netting set.
    Se asume que netting_set_df contiene solo trades AIR (asset_class == 'IR') y las columnas necesarias
    """
    # 1) calcular adjusted notional por trade
    trades = netting_set_df.copy()
    trades['adjusted_notional'] = trades.apply(lambda r: trade_adjusted_notional(r), axis=1)
    # 2) calcular delta supervisoria
    trades['supervisory_delta'] = trades.apply(lambda r: supervisory_delta(r), axis=1)

    # 3) calcular maturity factors por trade dependiendo si margined/unmargined
    margined = params_per_netting_set.get('margined', False)
    frequency_days = params_per_netting_set.get('margin_frequency_days', None)
    MPOR = compute_mpor_from_margin_frequency(frequency_days) if margined else None
    trades['MF'] = trades.apply(lambda r: maturity_factor_margined(MPOR) if margined else maturity_factor_unmargined(r.get('maturity_years', 0.0)), axis=1)

    # 4) agrupar por currency (hedging set) y bucket
    trades['bucket'] = trades['end_years'].apply(assign_ir_bucket)
    # usamos la formula: D_jk = sum_{trades in currency j and bucket k} adjusted_notional * delta * MF
    grouped = trades.groupby(['currency', 'bucket']).apply(lambda df: (df['adjusted_notional'] * df['supervisory_delta'] * df['MF']).sum()).reset_index()
    grouped.columns = ['currency', 'bucket', 'D_k']

    # 5) para cada hedging set (currency) agregamos across buckets usando coeficientes (parrafo 169):
    # Effective notional per hedging set (currency j):
    # D_j = sqrt( D1^2 + D2^2 + D3^2 + 1.4*D1*D2 + 1.4*D2*D3 + 0.6*D1*D3 )
    result = {}
    for currency, grp in grouped.groupby('currency'):
        D1 = float(grp.loc[grp['bucket'] == 1, 'D_k'].sum())
        D2 = float(grp.loc[grp['bucket'] == 2, 'D_k'].sum())
        D3 = float(grp.loc[grp['bucket'] == 3, 'D_k'].sum())
        term = (D1**2 + D2**2 + D3**2 + 1.4*D1*D2 + 1.4*D2*D3 + 0.6*D1*D3)
        effective_notional = math.sqrt(max(term, 0.0))
        addon = SUPERVISORY_FACTORS['IR'] * effective_notional
        result[currency] = {
            'D1': D1, 'D2': D2, 'D3': D3,
            'effective_notional': effective_notional,
            'addon': addon
        }
    return result

# ADD-ON FX

def fx_addon(netting_set_df, params_per_netting_set, domestic_currency='USD'):
    trades = netting_set_df.copy()
    # se asume que existe 'foreign_leg_notional_dom' columna con notional convertido a moneda domestica
    trades['adjusted_notional'] = trades.apply(lambda r: trade_adjusted_notional(r), axis=1)
    trades['supervisory_delta'] = trades.apply(lambda r: supervisory_delta(r), axis=1)
    margined = params_per_netting_set.get('margined', False)
    frequency_days = params_per_netting_set.get('margin_frequency_days', None)
    MPOR = compute_mpor_from_margin_frequency(frequency_days) if margined else None
    trades['MF'] = trades.apply(lambda r: maturity_factor_margined(MPOR) if margined else maturity_factor_unmargined(r.get('maturity_years', 0.0)), axis=1)

    # hedging set for FX is currency pair; group by currency pair (assume 'currency_pair' col)
    if 'currency_pair' not in trades.columns:
        trades['currency_pair'] = trades['currency']  # fallback

    grouped = trades.groupby('currency_pair').apply(lambda df: (df['adjusted_notional'] * df['supervisory_delta'] * df['MF']).sum()).reset_index()
    grouped.columns = ['currency_pair', 'effective_notional']
    grouped['addon'] = grouped['effective_notional'].abs() * SUPERVISORY_FACTORS['FX']

    return grouped.set_index('currency_pair')['addon'].to_dict()

# ADD-ON CREDIT

def credit_addon(netting_set_df, params_per_netting_set):
    trades = netting_set_df.copy()
    trades['adjusted_notional'] = trades.apply(lambda r: trade_adjusted_notional(r), axis=1)
    trades['supervisory_delta'] = trades.apply(lambda r: supervisory_delta(r), axis=1)
    trades['MF'] = trades.apply(lambda r: maturity_factor_unmargined(r.get('maturity_years', 0.0)), axis=1)  # por simplicidad

    # entity-level effective notional
    # se asume columna 'reference_entity' y 'credit_rating' o indication IG/SG
    trades['entity'] = trades.get('reference_entity', trades.get('reference_index', 'UNKNOWN'))
    trades['entity_effective_notional'] = trades['adjusted_notional'] * trades['supervisory_delta'] * trades['MF']
    entity_sum = trades.groupby('entity')['entity_effective_notional'].sum().reset_index()
    # entity add-on = SF(entity) * abs(effective_notional)
    addons = []
    for _, r in entity_sum.iterrows():
        ent = r['entity']
        eff = float(r['entity_effective_notional'])
        # seleccionar SF
        # si es index se espera columna reference_index_rating IG/SG
        # fallback: usar single name mapping si la columna 'credit_rating' existe
        sf = None
        # intentar recuperar rating
        sample_row = trades[trades['entity'] == ent].iloc[0]
        rating = sample_row.get('credit_rating', None)
        if rating in SUPERVISORY_FACTORS['Credit_single']:
            sf = SUPERVISORY_FACTORS['Credit_single'][rating]
        else:
            # si es index IG/SG
            idx_type = sample_row.get('index_grade', None)
            if idx_type in SUPERVISORY_FACTORS['Credit_index']:
                sf = SUPERVISORY_FACTORS['Credit_index'][idx_type]
            else:
                # fallback conservador
                sf = 0.0106
        addon_ent = abs(eff) * sf
        addons.append({'entity': ent, 'eff_not': eff, 'sf': sf, 'addon': addon_ent})

    addon_df = pd.DataFrame(addons)
    # aplicar la fórmula de agregación con componente sistemático e idiosincrático (parrafo 173-174):
    # systematic = sum(addon_k * r_k)
    # idiosyncratic = sum( (addon_k)^2 * (1 - r_k^2) )
    # total_addon = sqrt( systematic^2 + idiosyncratic )
    systematic = 0.0
    idiosyncratic = 0.0
    for _, r in addon_df.iterrows():
        addon_k = r['addon']
        # r_k: correlation parameter (0.5 para single-name, 0.8 para indices)
        sample_row = trades[trades['entity'] == r['entity']].iloc[0]
        rating = sample_row.get('credit_rating', '')
        if rating in (None, ''):
            r_k = 0.5
        else:
            # identificar si index
            if rating in ('IG', 'SG'):
                r_k = 0.8
            else:
                r_k = 0.5
        systematic += addon_k * r_k
        idiosyncratic += (addon_k**2) * (1 - r_k**2)
    total_addon = math.sqrt(systematic**2 + idiosyncratic)
    return float(total_addon)

# ADD-ON Equity & Commodity

def generic_single_factor_addon(netting_set_df, asset_type):
    """Asset_type: 'Equity' o 'Commodity' - aplica la formula single-factor (parrafos 176 y 179)
    Se asume que netting_set_df tiene columna 'entity' (equity ticker o commodity type)
    """
    trades = netting_set_df.copy()
    trades['adjusted_notional'] = trades.apply(lambda r: trade_adjusted_notional(r), axis=1)
    trades['supervisory_delta'] = trades.apply(lambda r: supervisory_delta(r), axis=1)
    trades['MF'] = trades.apply(lambda r: maturity_factor_unmargined(r.get('maturity_years', 0.0)), axis=1)
    trades['entity'] = trades.get('reference_entity', trades.get('entity', 'UNKNOWN'))
    trades['entity_eff_not'] = trades['adjusted_notional'] * trades['supervisory_delta'] * trades['MF']
    entity_sum = trades.groupby('entity')['entity_eff_not'].sum().reset_index()

    # calcular entity-level add-ons y luego agregar con correlaciones
    addons = []
    for _, r in entity_sum.iterrows():
        ent = r['entity']
        eff = float(r['entity_eff_not'])
        if asset_type == 'Equity':
            # determinar si index (se asume columna is_index)
            # campo is_index en las filas originales
            sample_row = trades[trades['entity'] == ent].iloc[0]
            is_index = sample_row.get('is_index', False)
            sf = SUPERVISORY_FACTORS['Equity_index'] if is_index else SUPERVISORY_FACTORS['Equity_single']
            rho = CORRELATION_PARAMS['Equity_index'] if is_index else CORRELATION_PARAMS['Equity_single']
        else:  # Commodity
            sample_row = trades[trades['entity'] == ent].iloc[0]
            typ = sample_row.get('commodity_type', 'Other')
            sf = SUPERVISORY_FACTORS['Commodity'].get(typ, 0.18)
            rho = CORRELATION_PARAMS['Commodity_hedging_set']
        addon_ent = abs(eff) * sf
        addons.append({'entity': ent, 'eff': eff, 'sf': sf, 'addon': addon_ent, 'rho': rho})

    addon_df = pd.DataFrame(addons)
    # Agregación single-factor: total = sqrt( (sum(addon_k * rho_k))^2 + sum(addon_k^2 * (1 - rho_k^2)) )
    systematic = 0.0
    idiosyncratic = 0.0
    for _, r in addon_df.iterrows():
        systematic += r['addon'] * r['rho']
        idiosyncratic += (r['addon']**2) * (1 - r['rho']**2)
    total_addon = math.sqrt(systematic**2 + idiosyncratic)
    return float(total_addon)


In [11]:
# %%
# FUNCTION to compute aggregate add-on across all asset classes for a netting set

def aggregate_addon_for_netting_set(df_netting, collateral_params):
    """df_netting: DataFrame con todas las trades del netting set
       collateral_params: dict con parametros del netting set que afectan MF (margined, frequency)
    """
    agg = 0.0
    # INTEREST RATE
    df_ir = df_netting[df_netting['asset_class'] == 'IR']
    if not df_ir.empty:
        ir_res = interest_rate_addon(df_ir, collateral_params)
        # ir_res es dict por currency
        ir_total = sum(v['addon'] for v in ir_res.values())
        agg += ir_total
    # FX
    df_fx = df_netting[df_netting['asset_class'] == 'FX']
    if not df_fx.empty:
        fx_res = fx_addon(df_fx, collateral_params)
        fx_total = sum(fx_res.values())
        agg += fx_total
    # Credit
    df_credit = df_netting[df_netting['asset_class'] == 'Credit']
    if not df_credit.empty:
        credit_total = credit_addon(df_credit, collateral_params)
        agg += credit_total
    # Equity
    df_eq = df_netting[df_netting['asset_class'] == 'Equity']
    if not df_eq.empty:
        eq_total = generic_single_factor_addon(df_eq, 'Equity')
        agg += eq_total
    # Commodity
    df_com = df_netting[df_netting['asset_class'] == 'Commodity']
    if not df_com.empty:
        com_total = generic_single_factor_addon(df_com, 'Commodity')
        agg += com_total

    return float(agg)

In [12]:
# MULTIPLIER (parrafo 148-149)
# multiplier = min(1, Floor + (1 - Floor) * exp( (V - C) / (2 * (1 - Floor) * AddOn_aggregate) ) )
# se tratan los casos con AddOn_aggregate == 0

def multiplier_from_VC_and_addon(V, C, addon_agg, floor=FLOOR):
    delta_VC = V - C
    if addon_agg <= 0:
        return 1.0 if delta_VC >= 0 else floor
    denom = 2.0 * (1.0 - floor) * addon_agg
    expo = (delta_VC) / denom
    m = floor + (1.0 - floor) * math.exp(expo)
    return min(1.0, m)

In [13]:

# FUNCIÓN PRINCIPAL: calcular EAD para un netting set

def calculate_EAD_for_netting_set(df_all_trades, collateral_params, alpha=ALPHA):
    """df_all_trades: DataFrame con todas las trades (varios netting sets posibles)
       collateral_params: dict con las entradas por netting_set_id -> dict de parametros de colateral
    """
    results = []
    for ns, group in df_all_trades.groupby('netting_set_id'):
        params = collateral_params.get(ns, {})
        # RC:
        RC = replacement_cost(group, params)
        # Aggregate AddOn (PFE component):
        addon = aggregate_addon_for_netting_set(group, params)
        # V and C for multiplier
        V = group['market_value'].sum()
        C = float(params.get('collateral_value', 0.0))
        mult = multiplier_from_VC_and_addon(V, C, addon)
        EAD = alpha * (RC + mult * addon)
        results.append({'netting_set_id': ns, 'RC': RC, 'AddOn': addon, 'Multiplier': mult, 'EAD': EAD})
    return pd.DataFrame(results)

In [14]:
# EJEMPLO / COMPROBACIÓN: Reproducir el Ejemplo 1 del Annex (Netting set 1)
# Datos tomados del ejemplo del documento para validar la implementación.

example_trades = [
    # Trade 1: IR swap 10 years USD 10,000 Fixed vs Floating market value +30
    {'netting_set_id': 'NS1', 'trade_id': 'T1', 'asset_class': 'IR', 'notional': 10000.0, 'currency': 'USD',
     'start_years': 0.0, 'end_years': 10.0, 'maturity_years': 10.0, 'is_option': False, 'position': 'long', 'market_value': 30.0},
    # Trade 2: IR swap 4 years USD 10,000 Floating vs Fixed market value -20
    {'netting_set_id': 'NS1', 'trade_id': 'T2', 'asset_class': 'IR', 'notional': 10000.0, 'currency': 'USD',
     'start_years': 0.0, 'end_years': 4.0, 'maturity_years': 4.0, 'is_option': False, 'position': 'short', 'market_value': -20.0},
    # Trade 3: swaption 1 into 10 years EUR 5,000 market value +50
    # Para el ejemplo usaremos datos equivalentes convirtiendo la notional a USD (en el doc todos en USD)
    {'netting_set_id': 'NS1', 'trade_id': 'T3', 'asset_class': 'IR', 'notional': 5000.0, 'currency': 'EUR',
     'start_years': 1.0, 'end_years': 11.0, 'maturity_years': 11.0, 'is_option': True, 'option_type': 'call',
     'option_forward_price': 0.06, 'option_strike': 0.05, 'option_T': 1.0, 'position': 'long', 'market_value': 50.0}
]

df_example = pd.DataFrame(example_trades)
# En el Ejemplo del documento todas las cantidades ya estaban en USD. Para reproducirlo exactamente, indicar currency = USD
# fijamos currency a USD para que las agrupaciones coincidan con el ejemplo
# en la práctica se deben convertir a moneda doméstica antes de ejecutar

df_example['currency'] = 'USD'

# Parámetros de colateral para NS1: netting set no margined (unmargined)
coll_params = {
    'NS1': {'collateral_value': 0.0, 'threshold': 0.0, 'mta': 0.0, 'nica': 0.0, 'margined': False}
}

res = calculate_EAD_for_netting_set(df_example, coll_params)
print('Resultado (Ejemplo 1):')
print(res.to_string(index=False))

# %%
# Guardar funciones / utilidades: exportar resultados detallados por netting set (opcional)

def detailed_report(df_all_trades, collateral_params):
    """Genera un reporte detallado por netting set: adjusted notional por trade, deltas, MF, RC, addons, multiplier y EAD."""
    results = []
    for ns, group in df_all_trades.groupby('netting_set_id'):
        params = collateral_params.get(ns, {})
        RC = replacement_cost(group, params)
        addon = aggregate_addon_for_netting_set(group, params)
        V = group['market_value'].sum()
        C = float(params.get('collateral_value', 0.0))
        mult = multiplier_from_VC_and_addon(V, C, addon)
        EAD = ALPHA * (RC + mult * addon)
        results.append({'netting_set_id': ns, 'RC': RC, 'AddOn': addon, 'Multiplier': mult, 'EAD': EAD})
    return pd.DataFrame(results)

# EOF


Resultado (Ejemplo 1):
netting_set_id   RC      AddOn  Multiplier        EAD
           NS1 60.0 422.567346         1.0 675.594284


C:\Users\djimenez\AppData\Local\Temp\ipykernel_15940\379639700.py:24: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = trades.groupby(['currency', 'bucket']).apply(lambda df: (df['adjusted_notional'] * df['supervisory_delta'] * df['MF']).sum()).reset_index()
